## Programación Entera
### Estrategia Neutral con Opciones
En este ejemplo se plantea una estrategia nuetral con opciones, vendiendo una cesta de opciones que se cubra con una compra de la opción sobre el índice correspondiente.  El inconveniente para el IBEX es que no todos los valores tienen opciones cotizando y además los multiplicadores de las opciones no nos permiten replicar exactamente la exposición al índice.  Por tando el objetivo consiste en **determinar el número de contratos a vender de cada opción disponible, de modo que se reproduzca el nominal contratado en la opción sobre el índice**


In [ ]:
import sys
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

### Datos
Datos de mercado del IBEX35 y de sus componentes

In [ ]:
import pickle
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)
with open('../data/benchmark.pkl', 'rb') as handle:
    benchmark = pickle.load(handle)

In [ ]:
close_dict = {tk: df.close for tk, df in stock_data.items()}
close_df = pd.DataFrame(close_dict)

In [ ]:
close_year = close_df.loc['2019-09-02':'2019-12-31']
stock_close = close_year.dropna(axis=1)

In [ ]:
bm_year = benchmark['ibex'].close

Tenemos la lista de acciones que tienen cotizando 
opciones en MEFF

In [ ]:
liquid_tk = ['SAN', 'BBVA', 'ITX', 'TEF', 'REP',
             'IBE', 'CLNX', 'AMS', 'ELE', 'IAG',
             'FER', 'GRF', 'REE', 'ENG', 'NTGY',
             'ACS', 'CABK', 'ACX']

In [ ]:
ibd = benchmark['ibex'].close

seleccionamos solo los precios de cierre de las acciones 
sobre las que vamos a trabajar. Haremos la estimación
utilizando los últimos 3 meses del 2019

In [ ]:
liquid_close = stock_close[liquid_tk]
liquid_close.tail()

Una opción sobre el índice tiene multiplicador 1, por lo que utilizaremos un factor sobre 
el precio del IBEX para representar el nominal total sobre el que queremos trabajar
Ver detalles en la descripción del [opciones sobre el IBEX35](https://www.meff.es/esp/Derivados-Financieros/Opciones-sobre-IBEX35)

In [ ]:
bm_factor = 10

In [ ]:
n_stocks = len(liquid_tk)
n_days = liquid_close.shape[0]
n_stocks, n_days

Las variables de decisión 
- Numero de contratos de cada opción
- variables de error de cada día que se deben minimizar

In [ ]:
var_contract = cp.Variable(n_stocks, integer=True)
var_epsilon = cp.Variable(n_days)

Restricciones
- el nominal de los contratos de opciones de la cesta menos de los contratos sobre el índice debe ser igual al error del día
- la cantidad de contratos no puede ser negativa

In [ ]:
day_constraints = []
i = 0
for iday, iprices in liquid_close.iterrows():
    i_constraint = (var_contract @ (iprices.values*100)) - ibd.loc[iday]*bm_factor == var_epsilon[i]
    day_constraints = day_constraints + [i_constraint]
    i += 1 

In [ ]:
constraints = [ var_contract >= 0]
constraints.extend(day_constraints)

Minimizamos la suma de los valores absolutos de los errores

In [ ]:
objective = cp.Minimize(cp.sum(cp.abs(var_epsilon)))

In [ ]:
prob = cp.Problem(objective, constraints)

In [ ]:
prob.solve()

Solución

In [ ]:
solucion = pd.Series(var_contract.value, index=liquid_close.columns)
solucion.round(2)

In [ ]:
all_nominal = (solucion * liquid_close * 100)
diff_nominal = all_nominal.sum(axis=1) - ibd*bm_factor
diff_nominal.plot()